In [30]:
import pandas as pd
import numpy as np
import datetime
import os


In [31]:
all_sales_file=os.listdir("/home/jian/Projects/Smoothie_King/Weekly_Sales/")
df_all_sales_file=pd.DataFrame({"sales_file":all_sales_file})
df_all_sales_file['sales_file']="/home/jian/Projects/Smoothie_King/Weekly_Sales/"+df_all_sales_file['sales_file']
df_all_sales_file['date_received']=df_all_sales_file['sales_file'].apply(lambda x: x.split("WeeklySalesSummaryData_")[1][:8])
df_all_sales_file=df_all_sales_file.sort_values("date_received",ascending=False)
df_recent_4_sales_file=df_all_sales_file.head(4)
df_recent_4_sales_file

,sales_file,date_received
20,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190603
19,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190527
18,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190520
17,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190513


In [32]:
recent_week_sales=pd.read_excel(df_recent_4_sales_file['sales_file'].tolist()[0],dtype=str)
recent_week_sales['NetSalesSmoothie']=recent_week_sales['NetSalesSmoothie'].astype(float)
recent_week_sales['NetSalesRetail']=recent_week_sales['NetSalesRetail'].astype(float)
recent_week_sales=recent_week_sales[(recent_week_sales['NetSalesSmoothie']>0) | (recent_week_sales['NetSalesRetail']>0)]

recent_1_week_stores=recent_week_sales['StoreNumber'].unique().tolist()

In [33]:
recent_4_weeks_sales=pd.DataFrame()
for file in df_recent_4_sales_file['sales_file'].tolist():
    df=pd.read_excel(file,dtype=str)
    df['NetSalesSmoothie']=df['NetSalesSmoothie'].astype(float)
    df['NetSalesRetail']=df['NetSalesRetail'].astype(float)
    df=df[(df['NetSalesSmoothie']>0) | (df['NetSalesRetail']>0)]
    recent_4_weeks_sales=recent_4_weeks_sales.append(df)
recent_4_week_stores=recent_4_weeks_sales.groupby("StoreNumber")['NetSalesSmoothie'].count().to_frame().reset_index()
recent_4_week_stores=recent_4_week_stores[recent_4_week_stores['NetSalesSmoothie']>=3]
recent_4_week_stores=recent_4_week_stores['StoreNumber'].unique().tolist()

In [34]:
inclusion_stores=list(set(recent_1_week_stores+recent_4_week_stores))
len(inclusion_stores)

919

In [35]:
recent_store_df=pd.read_excel("/home/jian/Projects/Smoothie_King/store_list_weekly/StoreList_20190603.xlsx",dtype=str)
recent_store_df=recent_store_df[recent_store_df['status']!="Closed"]
recent_store_df=recent_store_df[recent_store_df['storenumber'].isin(inclusion_stores)].drop_duplicates()
recent_store_df.shape

# recent_store_df.groupby("storenumber")['city'].count().to_frame().reset_index().sort_values("city",ascending=False).head(4)

(919, 14)

In [36]:
store_GPS_verified=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/QC_address/SK_store_listed_verified_JL_2019-04-29.xlsx",sheetname="assigned_lat_long",dtype=str)
new_stores_to_google=recent_store_df[~recent_store_df['storenumber'].isin(store_GPS_verified['storenumber'].tolist())]
new_stores_to_google=new_stores_to_google.reset_index()
del new_stores_to_google['index']

In [37]:
google_update_lat_lng={}
google_update_lat_lng.update({"1634":[38.5392912,-91.0126167]})
google_update_lat_lng.update({"1691":[33.5507844,-84.2754112]})
google_update_lat_lng.update({"1770":[39.0850624,-84.4783431]})
google_update_lat_lng.update({"1740":[29.8639979,-95.7214522]})
google_update_lat_lng.update({"1791":[30.6323826,-97.6931853]})
google_update_lat_lng.update({"1637":[39.9591837,-104.9937028]})
google_update_lat_lng.update({"1693":[33.9975027,-84.0911475]})
google_update_lat_lng.update({"1615":[32.6459654,-97.036669]})
len(google_update_lat_lng)

8

In [38]:
google_update_lat_lng

{'1615': [32.6459654, -97.036669],
 '1634': [38.5392912, -91.0126167],
 '1637': [39.9591837, -104.9937028],
 '1691': [33.5507844, -84.2754112],
 '1693': [33.9975027, -84.0911475],
 '1740': [29.8639979, -95.7214522],
 '1770': [39.0850624, -84.4783431],
 '1791': [30.6323826, -97.6931853]}

In [11]:
new_stores_to_google['storenumber'][0]

'1634'

In [13]:
from haversine import haversine
new_stores_to_google['dist']=np.nan
new_stores_to_google['google_lat']=np.nan
new_stores_to_google['google_long']=np.nan

for index,row in new_stores_to_google.iterrows():
    store_id=row['storenumber']
    google_location=google_update_lat_lng[store_id]
    client_location=[float(row['Latitude']),float(row['Longitude'])]
    dist=haversine(google_location,client_location)*1000
    new_stores_to_google.loc[index,'dist']=dist
    new_stores_to_google.loc[index,'google_lat']=google_location[0]
    new_stores_to_google.loc[index,'google_long']=google_location[1]
    
    

In [14]:

writer=pd.ExcelWriter("./new_stores_to_be_googled_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
new_stores_to_google.to_excel(writer,"new_stores",index=False)
writer.save()


In [19]:
new_stores_to_google.shape

(8, 14)

# delivery from output

In [60]:
recent_store_df.shape

(1048, 22)

In [61]:
table_output=pd.read_excel("/home/jian/Projects/Smoothie_King/store_list_weekly/checking_new_stores/SK_store_listed_verified_JL_2019-06-04.xlsx",
                          dtype=str)
print(table_output.shape)
print(table_output['storenumber'].nunique())

table_output.head(2)


(941, 20)
937


,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,QC_level,assigned_lat,assigned_long,assigned_source,updated_date,SOTF_Forced
0,0000,7215 Veterans Memorial Blvd.,Metairie,LA,70003,1973-07-07 00:00:00,30.00589291,-90.22051644,Franchised,Steve Shields,70.183.114.37,Y,Same,nan,POI_address_matched_in_30_meters,30.00589291,-90.22051644,Client_list,nan,nan
1,0002,4539 Veterans Blvd.,Metairie,LA,70006,1989-09-11 00:00:00,30.00581824,-90.18467266,Franchised,Steve Shields,108.65.143.116,N,Same,nan,POI_address_matched_in_30_meters,30.00581824,-90.18467266,Client_list,nan,nan


In [62]:
all_sales_file=os.listdir("/home/jian/Projects/Smoothie_King/Weekly_Sales/")
df_all_sales_file=pd.DataFrame({"sales_file":all_sales_file})
df_all_sales_file['sales_file']="/home/jian/Projects/Smoothie_King/Weekly_Sales/"+df_all_sales_file['sales_file']
df_all_sales_file['date_received']=df_all_sales_file['sales_file'].apply(lambda x: x.split("WeeklySalesSummaryData_")[1][:8])
df_all_sales_file=df_all_sales_file.sort_values("date_received",ascending=False)
df_recent_4_sales_file=df_all_sales_file.head(4)
df_recent_4_sales_file

,sales_file,date_received
20,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190603
19,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190527
18,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190520
17,/home/jian/Projects/Smoothie_King/Weekly_Sales...,20190513


In [63]:
df.head(3)

,storenumber,NetSalesSmoothie_2019-05-11,NetSalesRetail_2019-05-11
0,0000,18353.31,534.20
1,0002,14937.21,60.95
2,0003,15752.90,1905.70


In [64]:
recent_4_weeks_sales=pd.DataFrame()
recent_store_df=pd.read_excel("/home/jian/Projects/Smoothie_King/store_list_weekly/StoreList_20190603.xlsx",dtype=str)

for file in df_recent_4_sales_file['sales_file'].tolist():
    df=pd.read_excel(file,dtype=str)
    week_end_dt=df['WeekEndDate'].tolist()[0][:10]
    df['NetSalesSmoothie']=df['NetSalesSmoothie'].astype(float)
    df['NetSalesRetail']=df['NetSalesRetail'].astype(float)
    df=df[['StoreNumber','NetSalesSmoothie','NetSalesRetail']]
    df=df.rename(columns={"StoreNumber":"storenumber",
                          'NetSalesSmoothie':'NetSalesSmoothie'+"_"+week_end_dt,
                          'NetSalesRetail':'NetSalesRetail'+"_"+week_end_dt})
    
    table_output=pd.merge(table_output,df,on="storenumber",how="left")
    recent_store_df=pd.merge(recent_store_df,df,on="storenumber",how="left")

In [65]:
table_output=table_output.sort_values(["storenumber","SOTF_Forced"])
table_output=table_output.drop_duplicates("storenumber")
table_output['inclusion_as_0604']=table_output['storenumber'].apply(lambda x: x in inclusion_stores)
writer=pd.ExcelWriter("./SK_all_stores_with_labels_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
table_output.to_excel(writer,"all_stores_with_inclusion_label",index=False)
recent_store_df.to_excel(writer,"all_stores_last_week",index=False)
writer.save()

In [73]:
inc_0604=table_output[table_output['inclusion_as_0604']==True]['storenumber'].unique().tolist()
storeclient_0604=recent_store_df['storenumber'].unique().tolist()

[x for x in storeclient_0604 if x not in inc_0604]

['0129',
 '0173',
 '0453',
 '0487',
 '0490',
 '0500',
 '0524',
 '0526',
 '0537',
 '0799',
 '0845',
 '0853',
 '0899',
 '0905',
 '0911',
 '0915',
 '0102',
 '1039',
 '1040',
 '1050',
 '1055',
 '0562',
 '0583',
 '0658',
 '0709',
 '0721',
 '0773',
 '1015',
 '0429',
 '1220',
 '1227',
 '1121',
 '1126',
 '0678',
 '1124',
 '0544',
 '0563',
 '0600',
 '0534',
 '1018R',
 '0618',
 '0367',
 '1348',
 '0841',
 '1194',
 '0934',
 '1036',
 '0285',
 '1066',
 '0326',
 '0996',
 '0833',
 '0264',
 '0322',
 '1372',
 '1074',
 '0644',
 '1268',
 '0444',
 '0782',
 '0019',
 '0388',
 '0975',
 '0541',
 '1063',
 '1116',
 '0354',
 '0275',
 '0133',
 '0930',
 '0651',
 '0954',
 '1540',
 '0723',
 '1103',
 '0636',
 '0673',
 '0342',
 '1256',
 '0957',
 '0436',
 '0590',
 '0467',
 '0420',
 '1281',
 '0984',
 '0229',
 '0262CLS',
 '0262',
 '1041',
 '1344',
 '0198',
 '0982',
 '1197',
 '0632',
 '1064',
 '0555',
 '0521',
 '0686NT',
 '0961',
 '0194',
 '1248',
 '0161',
 '0514',
 '0717',
 '1012',
 '1013',
 '0964',
 '0665',
 '0712',
 '07

In [68]:
table_output[(table_output['inclusion_as_0604']==True) & (table_output['status']!="Closed")]['storenumber'].nunique()

919

In [70]:
table_output.shape

(937, 29)